In [72]:
from jupyter_dash import JupyterDash

In [73]:
import dash
import json
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image


In [74]:
teams = pd.read_csv("../data/team_stats.csv")
#teams.head()

In [75]:
team_ratings = teams[["Team", "ORtg", "DRtg", "Logo","Conference","Playoff"]]
team_ratings.head()

,Team,ORtg,DRtg,Logo,Conference,Playoff
0,Boston Celtics,114.4,106.9,celtics_logo.png,East,Playoff
1,Phoenix Suns,114.8,107.3,suns_logo.png,West,Playoff
2,Utah Jazz,116.7,110.5,jazz_logo.png,West,Playoff
3,Memphis Grizzlies,114.6,109.0,grizzlies_logo.png,West,Playoff
4,Golden State Warriors,112.5,106.9,warriors_logo.png,West,Playoff


In [76]:
app = JupyterDash(__name__)

#ratings_fig = px.scatter(team_ratings, x="ORtg", y="DRtg", hover_name="Team")

app.layout = html.Div(children=[
    html.H1(children='NBA Regular season 21-22'),

    html.P('Visualizing statistics from the 21-22 NBA regular season'),

    html.Div([
        html.Div([
            dcc.Graph(
                id='team-ratings-graph',
                clickData={'points': [{'hovertext': 'New York Knicks'}]}
            )
        ], style={'width': '75%', 'display': 'inline-block', 'padding': '0 20', 'verticalAlign': 'top'}),

        html.Div([
            html.P('Conference'),
            dcc.RadioItems(
                ['All', 'West', 'East'],
                'All',
                id='conference-filter',
                labelStyle={'padding': '1.5em'}
            ),

            html.P('Playoffs'),
            dcc.RadioItems(
                ['All', 'Playoff', 'Lottery'],
                'All',
                id='playoff-filter',
                labelStyle={'padding': '1.5em'}
            ),

            html.P('Division'),
            dcc.Dropdown(
                ['Atlantic', 'Central', 'Southeast', 'Pacific'],
                id='conference-filter'
            )
        ], style={'display': 'inline-block', 'width': '25%', 'padding': '5% 0', 'verticalAlign': 'top'})
    ]),

    html.H2(id='click-data'),

])

@app.callback(
    Output('team-ratings-graph', 'figure'),
    Input('conference-filter', 'value'),
    Input('playoff-filter', 'value')
)
def update_graph(conference, playoff):
    if conference == 'All':
        team_ratings_filtered = team_ratings
    else:
        team_ratings_filtered = team_ratings[team_ratings['Conference'] == conference]

    if playoff == 'All':
        team_ratings_filtered = team_ratings_filtered
    else:
        team_ratings_filtered = team_ratings_filtered[team_ratings_filtered['Playoff'] == playoff]

    ratings_fig = px.scatter(team_ratings_filtered, x="ORtg", y="DRtg", hover_name="Team")

    for i, row in team_ratings_filtered.iterrows():
        logo = row["Logo"]
        ratings_fig.add_layout_image(
            source=Image.open(f"../data/logos/{logo}"),
            x=row['ORtg'],
            y=row['DRtg'],
            xref="x",
            yref="y",
            sizex=2.5,
            sizey=2.5,
            xanchor="center",
            yanchor="middle",
        )

    return ratings_fig

@app.callback(
    Output('click-data', 'children'),
    Input('team-ratings-graph', 'clickData'))
def display_click_data(clickData):
    return clickData["points"][0]["hovertext"]


In [77]:
app.run_server()

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.